# Checknews

> Plataforma web para detecção de fake news

### Bibliotecas básicas e outros imports

In [ ]:
"""
    Bibliotecas básicas de DS
"""
import warnings
import pandas as pd
import numpy as np


"""
 Bibliotecas para melhora da performance e análise do dataset
"""
import sidetable
from pandarallel import pandarallel
from deepchecks.tabular import Dataset
from deepchecks.tabular.suites import data_integrity, train_test_validation

"""
    Bibliotecas de visualização de dados
"""
import matplotlib.pyplot as plt
%matplotlib inline

"""
    Biblioteca MLOps
"""
import mlflow
import mlflow.sklearn

"""
    Bibliotecas de ML
"""
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, RocCurveDisplay, ConfusionMatrixDisplay, accuracy_score, recall_score, precision_score, f1_score, log_loss

"""
    Arquivo de pré-processamento
"""
%run modules/nlp/pre_processing/pre_processor.py
%run modules/nlp/data_handler.py

"""
    Configurações básicas
"""
warnings.filterwarnings('ignore')
pandarallel.initialize(progress_bar=True)

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
np.set_printoptions(suppress=True, precision=3)

### Dados para teste

In [ ]:
df = pd.read_excel('data/data_fake_news_final.xlsx')
df = df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')
df.head(1)

### Elimina colunas desnecessários e preenche nulos para origem

In [ ]:
df['origin'].fillna('origem desconhecida', inplace=True)

### Valores nulos

In [ ]:
df.stb.missing()

### Analisando a integridade dos dados

In [ ]:
ds = Dataset(df, label='label')

integ_suite = data_integrity()
suite_result = integ_suite.run(ds)
suite_result.show()

### Pré-processamento

In [ ]:
df['text'] = df['text'].parallel_apply(cleaning)
df['origin'] = df['origin'].parallel_apply(prepareData)
df.sample(2)

### Remove dados duplicados

In [ ]:
df.drop_duplicates(subset=['text', 'origin'], inplace=True)
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')

### Balanceamento dos dados

In [ ]:
df['label'].value_counts()

### Seleção das classes

In [ ]:
min_class_count = 16693
df_filtered = df.groupby('label').apply(lambda x: x.sample(min_class_count)).reset_index(drop=True)
df_filtered['label'].value_counts()

### Encoder dos dados

In [ ]:
y = labelEncoder(df_filtered['label'])

### Divisão base

In [ ]:
df_filtered.drop(['label'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = kFoldSplitTrainTest(df_filtered,
                                                       y,
                                                       ['text', 'origin'])

### Validando os dados de treino e testes

In [ ]:
train_ds = Dataset(X_train.reset_index(drop=True), label=y_train, cat_features=[])
test_ds = Dataset(X_test.reset_index(drop=True), label=y_test, cat_features=[])

validation_suite = train_test_validation()
suite_result = validation_suite.run(train_ds, test_ds)
suite_result

### MLOPS

In [ ]:
dataset = 'data_fake_news_final.xlsx'
version = 'v1'
title = 'experimentos_checknews'

#### Decision tree

In [ ]:
mlflow.set_experiment(title)
with mlflow.start_run(run_name='DecisionTreeClassifier'):
    mlflow.set_tags({'dataset': dataset, 'version': version})
    
    column_transformer = ColumnTransformer([
        ('vectorizer_col1', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 0),
        ('vectorizer_col2', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 1),
    ])

    criterion = 'gini'
    splitter = 'random'

    dt_classifier_pipeline = Pipeline([
        ('vectorizer', column_transformer),
        ('clf', CalibratedClassifierCV(DecisionTreeClassifier(criterion=criterion, splitter=splitter), method='isotonic', cv=5))
    ])

    mlflow.log_param('criterion', criterion)
    mlflow.log_param('splitter', splitter)


    dt_classifier_pipeline.fit(X_train.values, y_train)
    y_pred = dt_classifier_pipeline.predict(X_test)
    y_prob = dt_classifier_pipeline.predict_proba(X_test)

    accuracy, recall, precision, f1, log, auc = getMetrics(y_test, y_pred, y_prob)

    mlflow.log_metric('acuracia', accuracy)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('precisão', precision)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('log', log)
    mlflow.log_metric('auc', auc)
    
    confusionMatrix('Matriz de confusão', dt_classifier_pipeline, X_test, y_test, y.classes)
    plt.savefig('img/matriz_classifier_dt.png')

    print(getClassificationReport(y_test, y_pred, y.classes))
    
    curvaRoc('DecisionTreeClassifier', y_test, y_pred, auc)
    plt.savefig('img/curva_roc_dt.png')

    mlflow.log_artifact(local_path='img/matriz_classifier_dt.png', artifact_path='img')
    mlflow.log_artifact(local_path='data/'+dataset, artifact_path='data')
    mlflow.log_artifact(local_path='img/curva_roc_dt.png', artifact_path='img')
    mlflow.sklearn.log_model(dt_classifier_pipeline, 'model_dt')
    print('Modelo: ', mlflow.active_run().info.run_uuid)

#### SVC

In [ ]:
mlflow.set_experiment(title)
with mlflow.start_run(run_name='LinearSVC'):
    mlflow.set_tags({'dataset': dataset, 'version': version})

    column_transformer = ColumnTransformer([
        ('vectorizer_col1', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 0),
        ('vectorizer_col2', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 1),
    ])

    penalty = 'l2'
    loss = 'squared_hinge'

    svc_classifier_pipeline = Pipeline([
        ('vectorizer', column_transformer),
        ('clf', CalibratedClassifierCV(LinearSVC(penalty=penalty, loss=loss), method='isotonic', cv=5))
    ])

    mlflow.log_param('penalty', penalty)
    mlflow.log_param('loss', loss)

    svc_classifier_pipeline.fit(X_train.values, y_train)
    y_pred = svc_classifier_pipeline.predict(X_test)
    y_prob = svc_classifier_pipeline.predict_proba(X_test)

    accuracy, recall, precision, f1, log, auc = getMetrics(y_test, y_pred, y_prob)

    mlflow.log_metric('acuracia', accuracy)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('precisão', precision)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('log', log)
    mlflow.log_metric('auc', auc)

    confusionMatrix('Matriz de confusão', svc_classifier_pipeline, X_test, y_test, y.classes)
    plt.savefig('img/matriz_classifier_svc.png')

    print(getClassificationReport(y_test, y_pred, y.classes))
    
    curvaRoc('LinearSVC', y_test, y_pred, auc)
    plt.savefig('img/curva_roc_svc.png')

    mlflow.log_artifact(local_path='img/matriz_classifier_svc.png', artifact_path='img')
    mlflow.log_artifact(local_path='data/'+dataset, artifact_path='data')
    mlflow.log_artifact(local_path='img/curva_roc_svc.png', artifact_path='img')
    mlflow.sklearn.log_model(svc_classifier_pipeline, 'model_svc')
    print('Modelo: ', mlflow.active_run().info.run_uuid)

#### Naive Bayes

In [ ]:
mlflow.set_experiment(title)
with mlflow.start_run(run_name='MultinomialNB'):
    mlflow.set_tags({'dataset': dataset, 'version': version})

    column_transformer = ColumnTransformer([
        ('vectorizer_col1', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 0),
        ('vectorizer_col2', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 1),
    ])

    alpha = 1.0
    force_alpha = 'warn'

    nb_classifier_pipeline = Pipeline([
        ('vectorizer', column_transformer),
        ('clf', CalibratedClassifierCV(MultinomialNB(alpha=alpha, force_alpha=force_alpha), method='isotonic', cv=5))
    ])

    mlflow.log_param('alpha', alpha)
    mlflow.log_param('force_alpha', force_alpha)

    nb_classifier_pipeline.fit(X_train.values, y_train)
    y_pred = nb_classifier_pipeline.predict(X_test)
    y_prob = nb_classifier_pipeline.predict_proba(X_test)

    accuracy, recall, precision, f1, log, auc = getMetrics(y_test, y_pred, y_prob)

    mlflow.log_metric('acuracia', accuracy)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('precisão', precision)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('log', log)
    mlflow.log_metric('auc', auc)

    confusionMatrix('Matriz de confusão', nb_classifier_pipeline, X_test, y_test, y.classes)
    plt.savefig('img/matriz_classifier_nb.png')

    print(getClassificationReport(y_test, y_pred, y.classes))
    
    curvaRoc('MultinomialNB', y_test, y_pred, auc)
    plt.savefig('img/curva_roc_nb.png')

    mlflow.log_artifact(local_path='img/matriz_classifier_nb.png', artifact_path='img')
    mlflow.log_artifact(local_path='data/'+dataset, artifact_path='data')
    mlflow.log_artifact(local_path='img/curva_roc_nb.png', artifact_path='img')
    mlflow.sklearn.log_model(nb_classifier_pipeline, 'model_nb')
    print('Modelo: ', mlflow.active_run().info.run_uuid)

#### Random Forest

In [ ]:
mlflow.set_experiment(title)
with mlflow.start_run(run_name='RandomForestClassifier'):
    mlflow.set_tags({'dataset': dataset, 'version': version })
    
    column_transformer = ColumnTransformer([
        ('vectorizer_col1', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 0),
        ('vectorizer_col2', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 1),
    ])

    n_estimators = 300
    criterion = 'entropy'

    rf_classifier_pipeline = Pipeline([
        ('vectorizer', column_transformer),
        ('clf', CalibratedClassifierCV(RandomForestClassifier(n_estimators=n_estimators, criterion=criterion), method='isotonic', cv=5))
    ])

    mlflow.log_param('n_estimators', n_estimators)
    mlflow.log_param('criterion', criterion)

    rf_classifier_pipeline.fit(X_train.values, y_train)
    y_pred = rf_classifier_pipeline.predict(X_test)
    y_prob = rf_classifier_pipeline.predict_proba(X_test)

    accuracy, recall, precision, f1, log, auc = getMetrics(y_test, y_pred, y_prob)

    mlflow.log_metric('acuracia', accuracy)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('precisão', precision)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('log', log)
    mlflow.log_metric('auc', auc)

    confusionMatrix('Matriz de confusão', rf_classifier_pipeline, X_test, y_test, y.classes)
    plt.savefig('img/matriz_classifier_rf.png')

    print(getClassificationReport(y_test, y_pred, y.classes))

    curvaRoc('RandomForestClassifier', y_test, y_pred, auc)
    plt.savefig('img/curva_roc_rf.png')

    mlflow.log_artifact(local_path='img/matriz_classifier_rf.png', artifact_path='img')
    mlflow.log_artifact(local_path='data/'+dataset, artifact_path='data')
    mlflow.log_artifact(local_path='img/curva_roc_rf.png', artifact_path='img')
    mlflow.sklearn.log_model(rf_classifier_pipeline, 'model_rf')
    print('Modelo: ', mlflow.active_run().info.run_uuid)

#### MLP

In [ ]:
mlflow.set_experiment(title)
with mlflow.start_run(run_name='MLPClassifier'):
    mlflow.set_tags({'dataset': dataset, 'version': version })
    
    column_transformer = ColumnTransformer([
        ('vectorizer_col1', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 0),
        ('vectorizer_col2', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 1),
    ])

    hidden_layer_sizes = (100, 64, 32)
    max_iter = 1000

    mlp_classifier_pipeline = Pipeline([
        ('vectorizer', column_transformer),
        ('clf', CalibratedClassifierCV(MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, verbose=False), method='isotonic', cv=5))
    ])

    mlflow.log_param('hidden_layer_sizes', hidden_layer_sizes)
    mlflow.log_param('max_iter', max_iter)

    mlp_classifier_pipeline.fit(X_train.values, y_train)
    y_pred = mlp_classifier_pipeline.predict(X_test)
    y_prob = mlp_classifier_pipeline.predict_proba(X_test)

    accuracy, recall, precision, f1, log, auc = getMetrics(y_test, y_pred, y_prob)

    mlflow.log_metric('acuracia', accuracy)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('precisão', precision)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('log', log)
    mlflow.log_metric('auc', auc)

    confusionMatrix('Matriz de confusão', mlp_classifier_pipeline, X_test, y_test, y.classes)
    plt.savefig('img/matriz_classifier_mlp.png')

    print(getClassificationReport(y_test, y_pred, y.classes))

    curvaRoc('MLPClassifier', y_test, y_pred, auc)
    plt.savefig('img/curva_roc_mlp.png')

    mlflow.log_artifact(local_path='img/matriz_classifier_mlp.png', artifact_path='img')
    mlflow.log_artifact(local_path='data/'+dataset, artifact_path='data')
    mlflow.log_artifact(local_path='img/curva_roc_mlp.png', artifact_path='img')
    mlflow.sklearn.log_model(mlp_classifier_pipeline, 'model_mlp')
    print('Modelo: ', mlflow.active_run().info.run_uuid)

#### XGBC

In [ ]:
mlflow.set_experiment(title)
with mlflow.start_run(run_name='XGBClassifier'):
    mlflow.set_tags({'dataset': dataset, 'version': version })
    
    column_transformer = ColumnTransformer([
        ('vectorizer_col1', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 0),
        ('vectorizer_col2', TfidfVectorizer(stop_words = STOPWORDS, lowercase = False), 1),
    ])

    learning_rate = 0.5
    max_depth = 4
    min_child_weight = 8
    nthread = 1
    subsample = 0.8

    xgbc_classifier_pipeline = Pipeline([
        ('vectorizer', column_transformer),
        ('clf', CalibratedClassifierCV(XGBClassifier(learning_rate=learning_rate,
                                                     max_depth=max_depth,
                                                     min_child_weight=min_child_weight,
                                                     nthread=nthread,
                                                     subsample=subsample), method='isotonic', cv=5))
    ])

    mlflow.log_param('learning_rate', learning_rate)
    mlflow.log_param('max_depth', max_depth)
    mlflow.log_param('min_child_weight', min_child_weight)
    mlflow.log_param('nthread', nthread)
    mlflow.log_param('subsample', subsample)

    xgbc_classifier_pipeline.fit(X_train.values, y_train)
    y_pred = xgbc_classifier_pipeline.predict(X_test)
    y_prob = xgbc_classifier_pipeline.predict_proba(X_test)

    accuracy, recall, precision, f1, log, auc = getMetrics(y_test, y_pred, y_prob)

    mlflow.log_metric('acuracia', accuracy)
    mlflow.log_metric('recall', recall)
    mlflow.log_metric('precisão', precision)
    mlflow.log_metric('f1', f1)
    mlflow.log_metric('log', log)
    mlflow.log_metric('auc', auc)

    confusionMatrix('Matriz de confusão', xgbc_classifier_pipeline, X_test, y_test, y.classes)
    plt.savefig('img/matriz_classifier_xgbc.png')
    
    print(getClassificationReport(y_test, y_pred, y.classes))

    curvaRoc('XGBClassifier', y_test, y_pred, auc)
    plt.savefig('img/curva_roc_xgbc.png')

    mlflow.log_artifact(local_path='img/matriz_classifier_xgbc.png', artifact_path='img')
    mlflow.log_artifact(local_path='data/'+dataset, artifact_path='data')
    mlflow.log_artifact(local_path='img/curva_roc_xgbc.png', artifact_path='img')
    mlflow.sklearn.log_model(xgbc_classifier_pipeline, 'model_mlp')
    print('Modelo: ', mlflow.active_run().info.run_uuid)